In [702]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from datetime import date, timedelta
from scipy.stats.stats import spearmanr 
from scipy import stats
from pandas import *
import re
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
import re
plt.style.use(['seaborn-darkgrid'])

import warnings
warnings.filterwarnings('ignore')

# Preprocessing

In [703]:
# Set datetime index
# Filter out primary elections 
def make_pretty(CSV_list):
    clean_dfs = []
    for csv in CSV_list:
        df = read_csv(csv)
        df = df.drop(['Unnamed: 0'], axis=1)
        df['Date'] = to_datetime(df['Date'])
        df['Year'] = df['Year'].astype(int)
        df.columns = ['Race','Poll','Result','Spread','Date','Year']
        df = df[df['Race'].str.contains('Primary')==False]
        clean_dfs.append(df)
    return clean_dfs

clean_dfs = make_pretty(['RCP_governor.csv','RCP_senate.csv','RCP_house.csv'])
gov, sen, house = clean_dfs[0], clean_dfs[1], clean_dfs[2]

# This format is consistent for all dfs
gov.head()

,Race,Poll,Result,Spread,Date,Year
0,Colorado Governor - McInnis vs. Hickenlooper,Rasmussen Reports,"Hickenlooper 42, McInnis 48",McInnis +6,2010-04-16,2010
3,New York Governor - Levy vs. Cuomo,Siena,"Cuomo 58, Levy 23",Cuomo +35,2010-04-18,2010
4,New York Governor - Lazio vs. Cuomo,Siena,"Cuomo 61, Lazio 24",Cuomo +37,2010-04-18,2010
5,Texas Governor - Perry vs. White,Rasmussen Reports,"Perry 48, White 44",Perry +4,2010-04-19,2010
6,Massachusetts Governor - Baker vs. Patrick vs....,Western NE College,"Patrick 34, Baker 27, Cahill 29",Patrick +5,2010-04-19,2010


In [704]:
# Make the winners data pretty
winners = read_csv('This_Works.csv')
winners = winners.drop(['Unnamed: 0', 'index'], axis=1)
winners['Date'] = to_datetime(winners['Date'])
# Multiword states contain an underscore
winners['State'] = winners['State'].str.replace('_', ' ')
winners = winners.set_index('Date')
winners['LastName'] = [i[-1:][0] for i in winners['Name'].str.split()]
winners['JI'] = winners['LastName']+' '+winners['State']

# This is the part we care about in the aggregation
win = winners[['Party','JI']]
win.head()

,Party,JI
Date,,
2017-02-09,R,Strange Alabama
2018-01-03,D,Jones Alabama
2016-11-08,R,Shelby Alabama
2016-11-08,R,Murkowski Alaska
2016-11-08,R,McCain Arizona


In [705]:
# Helper function for classifying party
def party_class(sen_df):
    ind = []
    for i in sen_df['Party']:
        if i=='R':
            ind.append(0)
        elif i=='D':
            ind.append(1)
        else:
            ind.append(np.nan)
    return ind

## Senate

In [706]:
'''
Want dfs of similar form to the ones cleaned in the preprocessing section
Edge cases and errors explained below
'''
def Senate_Classification(sen_df, win_df):
    # JI (join index) is name and state -- this cuts down on errors down the line
    sen_df['Lead'], sen_df['State'] = [i[0] for i in sen_df['Spread'].str.split()], [i[:i.index('Senate')-1] for i in sen_df['Race']]
    sen_df['JI'] = sen_df['Lead']+' '+sen_df['State']
    
    # Join sen_df to win_df on JI
    sen_df = sen_df.set_index('JI').join(win_df.set_index('JI')).set_index('Date')
    sen_df = sen_df.reset_index().set_index(['Date', 'State'])
    
    # Democrat=1, Republican=0, nans stay as is
    ind = party_class(sen_df)
    sen_df['Party Class'] = ind
    
    return sen_df

sen_df = Senate_Classification(sen, win)
sen_df

,,Race,Poll,Result,Spread,Year,Lead,Party,Party Class
Date,State,,,,,,,,
2012-04-19,Missouri,Missouri Senate - Akin vs. McCaskill,Rasmussen Reports,"Akin 48, McCaskill 43",Akin +5,2012,Akin,R,0.0
2012-07-28,Missouri,Missouri Senate - Akin vs. McCaskill,Post-Dispatch/Mason-Dixon,"McCaskill 44, Akin 49",Akin +5,2012,Akin,R,0.0
2012-08-01,Missouri,Missouri Senate - Akin vs. McCaskill,Rasmussen Reports,"McCaskill 44, Akin 47",Akin +3,2012,Akin,R,0.0
2012-08-13,Missouri,Missouri Senate - Akin vs. McCaskill,SurveyUSA*,"McCaskill 40, Akin 51",Akin +11,2012,Akin,R,0.0
2012-08-21,Missouri,Missouri Senate - Akin vs. McCaskill,PPP (D),"McCaskill 43, Akin 44",Akin +1,2012,Akin,R,0.0
2012-09-18,Missouri,Missouri Senate - Akin vs. McCaskill,Gravis Marketing,"McCaskill 40, Akin 44",Akin +4,2012,Akin,R,0.0
2014-02-13,North Carolina,North Carolina Senate - Alexander vs. Hagan,PPP (D),"Alexander 45, Hagan 38",Alexander +7,2014,Alexander,NaN,NaN
2014-03-11,North Carolina,North Carolina Senate - Alexander vs. Hagan,PPP (D),"Alexander 45, Hagan 43",Alexander +2,2014,Alexander,NaN,NaN
2014-04-02,North Carolina,North Carolina Senate - Alexander vs. Hagan,SurveyUSA,"Alexander 46, Hagan 44",Alexander +2,2014,Alexander,NaN,NaN


In [707]:
# CSV Output
sen_df.to_csv('RCP_senate_DemRec.csv')

## Edge cases and errors
* This is the best way to aggregate the data, but there are obvious issues
* We have no way to classify the party of the individual leading a poll unless that individual has held office before (shows up in the winners data)
* We get nan for party if the poll results are tied -- this may be okay as is

**767 out of 4547 values are nans**

In [692]:
# Atwater doesn't have a party because he's never won an election
sen_df.reset_index().set_index('State').loc['Florida',].head()

,Date,Race,Poll,Result,Spread,Year,Lead,Party,Party Class
State,,,,,,,,,
Florida,2015-03-10,Florida Senate - Atwater vs. Wasserman Schultz,Mason-Dixon,"Atwater 45, Wasserman Schultz 35",Atwater +10,2015,Atwater,NaN,NaN
Florida,2015-03-10,Florida Senate - Atwater vs. Murphy,Mason-Dixon,"Atwater 46, Murphy 32",Atwater +14,2015,Atwater,NaN,NaN
Florida,2015-03-25,Florida Senate - Atwater vs. Grayson,PPP (D),"Atwater 41, Grayson 40",Atwater +1,2015,Atwater,NaN,NaN
Florida,2015-03-25,Florida Senate - Atwater vs. Murphy,PPP (D),"Atwater 41, Murphy 39",Atwater +2,2015,Atwater,NaN,NaN
Florida,2015-04-06,Florida Senate - Atwater vs. Grayson,Quinnipiac,"Atwater 42, Grayson 32",Atwater +10,2015,Atwater,NaN,NaN


In [693]:
# Here's our output when there's a tie
sen_df[sen_df['Result'].str.contains('Rubio')].tail()

,,Race,Poll,Result,Spread,Year,Lead,Party,Party Class
Date,State,,,,,,,,
2010-06-09,Florida,Florida Senate - Rubio vs. Meek vs. Crist,Rasmussen Reports,"Crist 37, Rubio 37, Meek 15",Tie,2010,Tie,NaN,NaN
2010-08-04,Florida,Florida Senate - Rubio vs. Greene vs. Crist,AIF/McLaughlin (R),"Crist 37, Rubio 37, Greene 16",Tie,2010,Tie,NaN,NaN
2016-06-29,Florida,Florida Senate - Rubio vs. Murphy,Bay News 9/SurveyUSA,"Rubio 43, Murphy 43",Tie,2016,Tie,NaN,NaN
2016-10-21,Florida,Florida Senate - Rubio vs. Murphy,FOX 13/Opinion Savvy,"Rubio 46, Murphy 46",Tie,2016,Tie,NaN,NaN
2016-10-30,Florida,Florida Senate - Rubio vs. Murphy,Gravis,"Rubio 46, Murphy 46",Tie,2016,Tie,NaN,NaN


In [694]:
# Certain states have senators with the same last names
# Joining on both name and state helped preven mismatches
print(win[win['JI'].str.contains('Scott')])
print(win[win['JI'].str.contains('Murphy')])

           Party                    JI
Date                                  
2016-11-08     R  Scott South Carolina
2016-11-08     R         Scott Georgia
2016-11-08     D         Scott Georgia
2016-11-08     D        Scott Virginia
2014-11-04     R         Scott Georgia
2014-11-04     D         Scott Georgia
2014-11-04     D        Scott Virginia
2012-11-06     R         Scott Georgia
2012-11-06     D         Scott Georgia
2012-11-06     D        Scott Virginia
2013-01-02     R  Scott South Carolina
2010-11-02     R         Scott Georgia
2010-11-02     D         Scott Georgia
2010-11-02     R  Scott South Carolina
2010-11-02     D        Scott Virginia
           Party                   JI
Date                                 
2016-11-08     D       Murphy Florida
2016-11-08     R  Murphy Pennsylvania
2014-11-04     D       Murphy Florida
2012-11-06     D   Murphy Connecticut
2012-11-06     D       Murphy Florida
2012-11-06     R  Murphy Pennsylvania
2010-11-02     D   Murphy Connect

## Logic and thought process

In [695]:
# Lead is the last name of the candidate currently in the lead
# JI (join index) is name and state -- this cuts down on errors down the line
sen['Lead'], sen['State'] = [i[0] for i in sen['Spread'].str.split()], [i[:i.index('Senate')-1] for i in sen['Race']]
sen['JI'] = sen['Lead']+' '+sen['State']

In [696]:
sen.head()

,Race,Poll,Result,Spread,Date,Year,Lead,State,JI
0,Florida Senate - Rubio vs. Meek vs. Crist,Quinnipiac,"Crist 32, Rubio 30, Meek 24",Crist +2,2010-04-15,2010,Crist,Florida,Crist Florida
1,Arkansas Senate - Boozman vs. Lincoln,R2000/Daily Kos (D),"Boozman 50, Lincoln 43",Boozman +7,2010-04-15,2010,Boozman,Arkansas,Boozman Arkansas
2,Arkansas Senate - Boozman vs. Halter,R2000/Daily Kos (D),"Boozman 48, Halter 41",Boozman +7,2010-04-15,2010,Boozman,Arkansas,Boozman Arkansas
3,Florida Senate - Rubio vs. Meek,Quinnipiac,"Rubio 42, Meek 38",Rubio +4,2010-04-15,2010,Rubio,Florida,Rubio Florida
6,Florida Senate - Crist vs. Meek,Quinnipiac,"Crist 48, Meek 34",Crist +14,2010-04-15,2010,Crist,Florida,Crist Florida


In [697]:
# Party added
sen = sen.set_index('JI').join(win.set_index('JI')).set_index('Date')
sen = sen.reset_index().set_index(['Date', 'State'])

In [698]:
winners = read_csv('This_Works.csv')
winners = winners.drop(['Unnamed: 0', 'index'], axis=1)
winners['Date'] = to_datetime(winners['Date'])
# Multiword states contain an underscore
winners['State'] = winners['State'].str.replace('_', ' ')
winners = winners.set_index('Date')
winners['LastName'] = [i[-1:][0] for i in winners['Name'].str.split()]
winners['JI'] = winners['LastName']+' '+winners['State']

# This is the part we care about in the aggregation
win = winners[['Party','JI']]
win.head()

,Party,JI
Date,,
2017-02-09,R,Strange Alabama
2018-01-03,D,Jones Alabama
2016-11-08,R,Shelby Alabama
2016-11-08,R,Murkowski Alaska
2016-11-08,R,McCain Arizona


In [699]:
# Democrat=1, Republican=2, nans stay as is
ind = []
for i in sen['Party']:
    if i=='R':
        ind.append(0)
    elif i=='D':
        ind.append(1)
    else:
        ind.append(np.nan)

sen['Party Class'] = ind

In [659]:
# Looks pretty good overall, but there are some repeated polls (check Oregon for example)
sen

,,Race,Poll,Result,Spread,Year,Lead,Party,Party Class
Date,State,,,,,,,,
2012-04-19,Missouri,Missouri Senate - Akin vs. McCaskill,Rasmussen Reports,"Akin 48, McCaskill 43",Akin +5,2012,Akin,R,0.0
2012-07-28,Missouri,Missouri Senate - Akin vs. McCaskill,Post-Dispatch/Mason-Dixon,"McCaskill 44, Akin 49",Akin +5,2012,Akin,R,0.0
2012-08-01,Missouri,Missouri Senate - Akin vs. McCaskill,Rasmussen Reports,"McCaskill 44, Akin 47",Akin +3,2012,Akin,R,0.0
2012-08-13,Missouri,Missouri Senate - Akin vs. McCaskill,SurveyUSA*,"McCaskill 40, Akin 51",Akin +11,2012,Akin,R,0.0
2012-08-21,Missouri,Missouri Senate - Akin vs. McCaskill,PPP (D),"McCaskill 43, Akin 44",Akin +1,2012,Akin,R,0.0
2012-09-18,Missouri,Missouri Senate - Akin vs. McCaskill,Gravis Marketing,"McCaskill 40, Akin 44",Akin +4,2012,Akin,R,0.0
2014-02-13,North Carolina,North Carolina Senate - Alexander vs. Hagan,PPP (D),"Alexander 45, Hagan 38",Alexander +7,2014,Alexander,NaN,NaN
2014-03-11,North Carolina,North Carolina Senate - Alexander vs. Hagan,PPP (D),"Alexander 45, Hagan 43",Alexander +2,2014,Alexander,NaN,NaN
2014-04-02,North Carolina,North Carolina Senate - Alexander vs. Hagan,SurveyUSA,"Alexander 46, Hagan 44",Alexander +2,2014,Alexander,NaN,NaN


## House

In [ ]:
# Much harder to get state names because we have no clear syntax in the race str
def House_Classification():
    for df in df_list:
        df['Lead'], df['State'] = [i[0] for i in df['Spread'].str.split()], [i[:i.index('[%1-9]')-1] for i in sen['Race']]
        sen['JI'] = sen['Lead']+' '+sen['State']

In [631]:
# I'm getting some errors in district
house['District'] = pd.DataFrame([pd.Series(i)[pd.Series(i).str.contains('\d')] for i in house['Race'].str.split()]).fillna('').sum(axis=1)
house.head()

,Race,Poll,Result,Spread,Date,Year,District
0,Pennsylvania 12th District Special Election - ...,PPP (D),"Burns 44, Critz 41",Burns +3,2010-04-20,2010,12th
1,New Hampshire 1st District - Guinta vs. Shea-P...,PPP (D),"Guinta 46, Shea-Porter 45",Guinta +1,2010-04-22,2010,1st
2,New Hampshire 2nd District - Bass vs. Swett,PPP (D),"Bass 47, Swett 32",Bass +15,2010-04-22,2010,2nd
3,New Hampshire 1st District - Guinta vs. Shea-P...,WMUR/UNH,"Guinta 42, Shea-Porter 38",Guinta +4,2010-04-30,2010,1st
4,New Hampshire 2nd District - Bass vs. Swett,WMUR/UNH,"Bass 44, Swett 27",Bass +17,2010-04-30,2010,2nd
